In [42]:
import pandas as pd
import tkinter as tk
import os
from tkinter import simpledialog
import csv
from datetime import datetime
from tkinter import ttk
from tkinter import messagebox
import matplotlib.pyplot as plt
from collections import defaultdict
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import numpy as np

In [53]:
def new_profile():
    # Create a new window for the user to input the number of budgets they want to create
    new_window = tk.Tk()
    new_window.geometry("500x300")
    new_window.title("New Profile")
    
    # Create a label and an entry field for the number of budgets
    label = tk.Label(new_window, text="How many budgets do you want to create?")
    label.pack(pady=10)
    
    entry = tk.Entry(new_window)
    entry.pack(pady=10)
    
    # Create a submit button that opens the window for entering budget information
    submit_button1 = tk.Button(new_window, text="Submit", command=lambda: open_budgets_window(new_window, int(entry.get())))
    submit_button1.pack(pady=10)

def open_budgets_window(root, num_budgets):
    
    # Create new window
    budget_window = tk.Toplevel(root)
    #budget_window.geometry("500x300")
    budget_window.title("Budgeting GUI - New Budgets")

    # Create a frame for the budgets
    budget_frame = tk.Frame(budget_window)
    budget_frame.pack(side="top", pady=10)

    # Create entries for each budget
    budget_entries = []
    for i in range(num_budgets):
        # Create a label and entry for the budget name
        name_label = tk.Label(budget_frame, text="Budget Name:")
        name_label.grid(row=i, column=0, padx=10, pady=10)
        name_entry = tk.Entry(budget_frame)
        name_entry.grid(row=i, column=1, padx=10, pady=10)

        # Create a label and entry for the budget amount
        amount_label = tk.Label(budget_frame, text="Budget Amount:")
        amount_label.grid(row=i, column=2, padx=10, pady=10)
        amount_entry = tk.Entry(budget_frame)
        amount_entry.grid(row=i, column=3, padx=10, pady=10)

        # Add the budget entries to the list
        budget_entries.append((name_entry, amount_entry))

    # Create a submit button
    submit_button = tk.Button(budget_frame, text="Submit", command=lambda: submit_budgets(budget_window, budget_entries))
    submit_button.grid(row=num_budgets, column=1, pady=10)

    # Create a button to submit the budgets
    def submit_budgets(budget_window, budget_entries):
        # Create list of budget tuples
        budgets = []
        for entry in budget_entries:
            name = entry[0].get().strip()
            amount_str = entry[1].get().strip()
            if not amount_str:
                amount = 0
            else:
                amount = int(amount_str)
            budgets.append((name, amount))
    
        # Prompt user for filename and save budgets CSV
        title = simpledialog.askstring("Save File", "Please enter the filename to save your budgets as (do not include file extension):", parent=budget_window)
        with open(title+'.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["budget", "amount"])
            writer.writerows(budgets)
    
        # Create purchases csv with columns for each budget
        purchases_title = title+'_purchases.csv'
        with open(purchases_title, 'w', newline='') as file:
            writer = csv.writer(file)
            headers = [budget[0] for budget in budgets]
            writer.writerow(headers)
    
        # close the budget window
        root.destroy()

    submit_button = tk.Button(budget_frame, text="Submit", command=lambda: submit_budgets(budget_window, budget_entries))
    submit_button.grid(row=num_budgets, column=1, pady=10)
    
    # Keep the budgets window open until it is closed
    budget_window.mainloop()

In [152]:
def get_budgets_from_csv(filename):  
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        headers = next(reader)
        return headers

def get_budget_files():
    files = os.listdir('.')
    budget_files = []
    for file in files:
        if file.endswith('.csv') and not file.endswith('_purchases.csv'):
            budget_files.append(file[:-4])
    return budget_files

def open_budget_window():
    budget_files = get_budget_files()
    
    if not budget_files:
        messagebox.showwarning("No budgets found", "No budgets found. Please create a budget first.")
        return

    budget_window = tk.Toplevel(root)
    budget_window.title("Select Budget")

    tk.Label(budget_window, text="Please select a budget to view or edit:").pack()

    selected_budget = tk.StringVar()
    selected_budget.set(budget_files[0])

    ttk.OptionMenu(budget_window, selected_budget, *budget_files).pack()

    def view_budget():
        budget_window.destroy()
        open_view_purchase_window(selected_budget.get())

    def add_purchase():
        budget_window.destroy()
        open_add_purchase_window(selected_budget.get())
        
    def edit_budget():
        budget_window.destroy()
        open_edit_budget_window(selected_budget.get())

    view_button = tk.Button(budget_window, text="View Budget", command=view_budget)
    view_button.pack(pady=10)

    add_button = tk.Button(budget_window, text="Add Purchase", command=add_purchase)
    add_button.pack(pady=10)
    
    edit_button = tk.Button(budget_window, text="Edit Budget", command=edit_budget)
    edit_button.pack(pady=10)

    cancel_button = tk.Button(budget_window, text="Cancel", command=budget_window.destroy)
    cancel_button.pack(pady=10)
    return


def open_view_purchase_window(budget_filename,date=None):
    budget_csv = budget_filename + '.csv'
    purchases_csv = budget_filename + '_purchases.csv'
    minibudgets = get_budgets_from_csv(purchases_csv)

    if not date:
        now = datetime.now()
        date = now.strftime("%m/%Y")

    # create the tkinter window
    view_window = tk.Toplevel()
    view_window.title("View Budget "+ date)
    
    # read the original csv file and store the information in a dictionary
    with open(budget_csv, "r") as budget_file:
        reader = csv.reader(budget_file)
        next(reader)  # skip header
        budget_dict = {row[0]: float(row[1]) for row in reader}
    
    # read the purchases csv file and store the information in a nested dictionary
    with open(purchases_csv, "r") as purchase_file:
        reader = csv.reader(purchase_file)
        df = pd.read_csv(purchase_file)
        purchases_dict = {}
        for minibudget in minibudgets:
            purchases_dict[minibudget] = []
            for _, entry in df[minibudget].iteritems():
                if pd.notnull(entry) and date in entry:
                    purchases_dict[minibudget].append(entry.split('|'))

    minibudget_spent_dict = {minibudgets[0]:0,
                            minibudgets[1]:0}
    # calculate the total spent for each minibudget
    for minibudget in minibudgets:
        total_spent = 0
        for entry in purchases_dict[minibudget]:
            total_spent += float(entry[0])
        minibudget_spent_dict[minibudget] = total_spent
    print(minibudget_spent_dict)

    # calculate the remaining amount for each minibudget
    remaining_dict = {}
    for minibudget, amount in budget_dict.items():
        remaining_dict[minibudget] = amount - minibudget_spent_dict[minibudget]
    
    # display the remaining amount for each minibudget in a tkinter Label
    tk.Label(view_window, text="Remaining Amount for Each Minibudget").grid(row=0, column=0, columnspan=2)
    for i, (minibudget, remaining_amount) in enumerate(remaining_dict.items()):
        tk.Label(view_window, text=f"{minibudget}: ${remaining_amount:.2f}").grid(row=i+1, column=1)
    
    colors = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02']
    grey_color = '#9e9e9e'
    
    # display the pie chart of the amount spent on each minibudget
    fig = Figure(figsize=(5, 5), dpi=100)
    ax = fig.add_subplot(111)
    colors_graph = []
    minibudget_labels = list(budget_dict.keys())
    minibudget_values = list(budget_dict.values())
    spent_amounts = list(minibudget_spent_dict.values())

    #unspent = sum(minibudget_values) - sum(spent_amounts)
    #minibudget_labels.append('Unspent')
    #spent_amounts.append(unspent)

    for i in range(len(spent_amounts)):
        minibudget_labels[i] += ' $' + str(spent_amounts[i])
        colors_graph.append(colors[1])

    fig, ax = plt.subplots(figsize=(8, 6))

    # Set the x-axis labels and positions
    x_pos = np.arange(len(minibudget_labels))
    ax.set_xticks(x_pos)
    ax.set_xticklabels(minibudget_labels)

    # Create the bars
    ax.bar(x_pos, spent_amounts, label='Spent',color=colors_graph)
    ax.bar(x_pos, minibudget_values, alpha=0.5, label='Budget',color=colors_graph)

    # Add the legend
    ax.legend()

    # Add the labels and title
    ax.set_xlabel('Minibudget')
    ax.set_ylabel('Dollars')
    ax.set_title('Spending Breakdown by Minibudget')
    
    
    canvas = FigureCanvasTkAgg(fig, master=view_window)
    canvas.draw()
    canvas.get_tk_widget().grid(row=0, column=2, rowspan=len(remaining_dict)+1)
    
    
    tk.Label(view_window, text="See a different month (MM/YYYY)").grid(row=len(remaining_dict.items())-2, column=0)
    new_date = tk.Entry(view_window)
    new_date.grid(row=len(remaining_dict.items())-1,column=0)
    
    def new_month():
        view_window.destroy
        open_view_purchase_window(budget_filename, date=new_date.get())
    
    new_date_button = tk.Button(view_window, text="View",command=new_month)
    new_date_button.grid(row=len(remaining_dict.items()),column=0)
    

def open_add_purchase_window(budget_filename):
    minibudgets = get_budgets_from_csv(budget_filename + '_purchases.csv')
    
    if not minibudgets:
        messagebox.showwarning("No budgets found", "No budgets found in file. Please create a budget first.")
        return

    purchase_window = tk.Toplevel(root)
    purchase_window.title("Add Purchase")

    
    tk.Label(purchase_window, text="Select minibudget:").grid(row=1, column=0)
    tk.Label(purchase_window, text="Enter amount ($):").grid(row=2, column=0)
    tk.Label(purchase_window, text="Enter description:").grid(row=3, column=0)
    tk.Label(purchase_window, text="Enter date (MM/YYYY):").grid(row=4,column=0)
    

    selected_minibudget = tk.StringVar()
    selected_minibudget.set(minibudgets[0])

    minibudget_dropdown = ttk.OptionMenu(purchase_window, selected_minibudget, *minibudgets)
    minibudget_dropdown.grid(row=1, column=1)
    amount = tk.Entry(purchase_window)
    amount.grid(row=2, column=1)
    description = tk.Entry(purchase_window)
    description.grid(row=3, column=1)
    now = datetime.now()
    default_text = tk.StringVar(value=now.strftime("%m/%Y"))
    date = tk.Entry(purchase_window,textvariable=default_text)
    date.grid(row=4, column=1)


    def add_purchase():
        info = list(['']*len(minibudgets))
        selected_index = minibudgets.index(selected_minibudget.get())
        info[selected_index] = str(amount.get())+'|'+str(date.get())+'|'+str(description.get())
        with open(budget_filename + '_purchases.csv', 'a', newline='') as purchase_file:
            writer = csv.writer(purchase_file)
            writer.writerow(info)
        messagebox.showinfo("Purchase added", f"Purchase of {amount.get()} from minibudget {selected_minibudget.get()} added.")
        purchase_window.destroy()
    
    add_button = tk.Button(purchase_window, text="Add", command=add_purchase)
    add_button.grid(row=5,column=0)

    
def open_edit_budget_window(budget_filename):
    budget_csv = budget_filename + '.csv'
    purchases_csv = budget_filename + '_purchases.csv'
    minibudgets = get_budgets_from_csv(purchases_csv)

    # create the tkinter window
    edit_window = tk.Toplevel()
    edit_window.title("Edit Budget")
    
    budget_df = pd.read_csv(budget_csv)
    minibudgets = list(budget_df['budget'].values)
    amounts = list(budget_df['amount'].values)
    
    # Create a frame for the budgets
    edit_frame = tk.Frame(edit_window)
    edit_frame.pack(side="top", pady=10)

    # Create entries for each budget
    budget_entries = []
    names = []
    for i in range(10):
        if i < len(minibudgets):
            default_name = tk.StringVar(value=minibudgets[i])
            default_amt = tk.StringVar(value=amounts[i])
        else:
            default_name = tk.StringVar(value=None)
            default_amt = tk.StringVar(value=None)

        # Create a label and entry for the budget name
        name_label = tk.Label(edit_frame, text="Budget Name:")
        name_label.grid(row=i, column=0, padx=10, pady=10)
        name_entry = tk.Entry(edit_frame,textvariable=default_name)
        name_entry.grid(row=i, column=1, padx=10, pady=10)

        # Create a label and entry for the budget amount
        amount_label = tk.Label(edit_frame, text="Budget Amount:")
        amount_label.grid(row=i, column=2, padx=10, pady=10)
        amount_entry = tk.Entry(edit_frame,textvariable=default_amt)
        amount_entry.grid(row=i, column=3, padx=10, pady=10)

        # Add the budget entries to the list
        budget_entries.append((name_entry, amount_entry))

    
    # Create a button to submit the budgets
    def submit_budgets(budget_window):
        # Create list of budget tuples
        print(len(budget_entries))
        budgets = []
        names = []
        for entry in budget_entries:
            name = entry[0].get().strip()
            amount_str = entry[1].get().strip()
            print(name, amount_str)
            if not amount_str:
                amount = 0
            else:
                amount = int(amount_str)
            if name and amount:
                budgets.append((name, amount))
                names.append(name)
        print('budgets', budgets)
        
        # Prompt user for filename and save budgets CSV
       
        with open(budget_filename+'.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["budget", "amount"])
            writer.writerows(budgets)
        
        new_rows = []
        new_names = []
        lost_names = []
        purchases_df = pd.read_csv(purchases_csv)
    
        for name in minibudgets:
            if name not in names:
                lost_names.append(name)

        purchases_df.drop(columns=lost_names,inplace=True)
        os.remove(purchases_csv)
        purchases_df.to_csv(purchases_csv,index=False)

        for name in names:
            if name not in minibudgets:
                new_names.append(name)

        with open(purchases_csv) as file:
            reader_obj = csv.reader(file)
            for i,row in enumerate(reader_obj):
                if i == 0:
                    print('row',row)
                    print('new_names',new_names)
                    new_rows.append(row + new_names)
                else:
                    new_rows.append(row + [] * len(new_names))
    
        os.remove(purchases_csv)
        with open(purchases_csv, 'w', newline='') as file:
            writer = csv.writer(file)
            print('new',new_rows)
            writer.writerows(new_rows)

        # close the budget window
        budget_window.destroy()
        
    # Create a submit button
    submit_button = tk.Button(edit_frame, text="Submit", command=lambda: submit_budgets(edit_window))
    submit_button.grid(row=11, column=1, pady=10)
    

In [153]:
def start_page():
    # Create the main window
    root = tk.Tk()
    root.geometry("500x300")
    root.title("Budgeting GUI")
    # Create a label
    label = tk.Label(root, text="Select whether you'd like to continue \n an existing budget or create a new profile:")
    label.pack(pady=10)
    # Create a frame for the buttons
    button_frame = tk.Frame(root)
    button_frame.pack(side="left", pady=10, fill="both", expand=True)

    # Add buttons to the frame
    button1 = tk.Button(button_frame, text="Create a new profile",command=new_profile)
    button1.place(relx=0.33, rely=0.4, anchor="center")

    button2 = tk.Button(button_frame, text="Continue existing",command= open_budget_window)
    button2.place(relx=0.66, rely=0.4, anchor="center")

    # Start the main event loop
    return root


root = start_page()
root.mainloop()

10
h 90
g 4
f 6
 
 
 
 
 
 
 
budgets [('h', 90), ('g', 4), ('f', 6)]
row ['h', 'g']
new_names ['f']
new [['h', 'g', 'f']]
10
h 90
g 4
 
 
 
 
 
 
 
 
budgets [('h', 90), ('g', 4)]
row ['h', 'g']
new_names []
new [['h', 'g']]
10
 
g 4
 
 
 
 
 
 
 
 
budgets [('g', 4)]
row ['g']
new_names []
new [['g']]
10
g 4
home decor 200
eyelash extensions 50
 
 
 
 
 
 
 
budgets [('g', 4), ('home decor', 200), ('eyelash extensions', 50)]
row ['g']
new_names ['home decor', 'eyelash extensions']
new [['g', 'home decor', 'eyelash extensions']]
10
g 4
 
eyelash extensions 50
 
 
 
 
 
 
 
budgets [('g', 4), ('eyelash extensions', 50)]
row ['g', 'eyelash extensions']
new_names []
new [['g', 'eyelash extensions'], ['', '']]


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/evefine/opt/anaconda3/lib/python3.8/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-2-baa314b1db24>", line 15, in <lambda>
    submit_button1 = tk.Button(new_window, text="Submit", command=lambda: open_budgets_window(new_window, int(entry.get())))
  File "<ipython-input-2-baa314b1db24>", line 85, in open_budgets_window
    budget_window.mainloop()
  File "/Users/evefine/opt/anaconda3/lib/python3.8/tkinter/__init__.py", line 1429, in mainloop
    self.tk.mainloop(n)
KeyboardInterrupt


In [149]:
budget_df = pd.read_csv('yes?.csv')
minibudgets = budget_df['budget'].values
amounts = budget_df['amount']
budget_df
list(minibudgets)

FileNotFoundError: [Errno 2] No such file or directory: 'yes?.csv'